In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

# titanic data

Najpierw zaloguj się do https://www.kaggle.com/ i przejdź do wyzwania https://www.kaggle.com/c/titanic, aby pobrać 
 * train.csv i test.csv. 

Zapisz je w katalogu datasets/titanic.

In [16]:
import os
TITANIC_PATH = os.path.join("datasets", "titanic")

In [17]:
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [18]:
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

* Dane są już podzielone na zestaw treningowy i zestaw testów. 
* Jednak dane testowe nie zawierają etykiet: Twoim celem jest wyszkolenie najlepszego modelu, który możesz wykorzystać w danych treningowych, następnie dokonanie swoich przewidywań na danych testowych i przesłanie ich do Kaggle, aby zobaczyć ostateczny wynik.

Rzućmy okiem na kilka pierwszych rzędów zestawu treningowego:

In [19]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


The attributes have the following meaning:

* Survived: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.
* Pclass: passenger class.
* Name, Sex, Age: self-explanatory
* SibSp: how many siblings & spouses of the passenger aboard the Titanic.
* Parch: how many children & parents of the passenger aboard the Titanic.
* Ticket: ticket id
* Fare: price paid (in pounds)
* Cabin: passenger's cabin number
* Embarked: where the passenger embarked the Titanic
* Let's get more info to see how much data is missing:

In [20]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Atrybuty **Age**, **Cabin** oraz **Embarked** są czasami zerowe (mniej niż 891 wartości bez wartości null), szczególnie w przypadku **Cabin** (77% ma wartość zerową). Zignorujemy teraz **Cabin** i skupimy się na reszcie. Atrybut **Age** ma około 19% wartości pustych, więc będziemy musieli zdecydować, co z nimi zrobić. Zastąpienie wartości null medianą wieku wydaje się uzasadnione.

Atrybuty **Name** i **Ticket** mogą mieć pewną wartość, ale będą one nieco trudne do przekształcenia w użyteczne liczby. Na razie będziemy je ignorować.

Rzućmy okiem na atrybuty liczbowe:

In [21]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


* Tylko 38% przeżyło: to wystarczająco blisko do 40%, więc **accuracy** będzie rozsądną miarą do oceny naszego modelu.

Sprawdźmy, czy etykiety przyjmują wartości 0 lub 1:

In [22]:
train_data["Survived"].unique()

array([0, 1], dtype=int64)

Nie zapomnij o etykietach:

In [23]:
X_train = train_data.drop(["Survived"], axis=1)
y_train = train_data["Survived"]

Teraz rzućmy okiem na wszystkie atrybuty kategoryczne:

In [24]:
print(X_train["Sex"].unique())
print(X_train["Embarked"].unique())

['male' 'female']
['S' 'C' 'Q' nan]


Atrybut **Embarked** mówi nam, gdzie pasażer zaokrętował: C = Cherbourg, Q = Queenstown, S = Southampton.

Teraz zbudujmy nasze **pipeline** preprocessingu. 

Wykorzystamy DataframeSelector aby wybrać określone atrybuty z DataFrame:

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin

# A class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

Zbudujmy **pipeline** dla atrybutów numerycznych:

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["Pclass", "Age", "SibSp", "Parch", "Fare"])),
        ("imputer", SimpleImputer(strategy="median")),
    ])

num_pipeline.fit(X_train)

Pipeline(steps=[('select_numeric',
                 DataFrameSelector(attribute_names=['Pclass', 'Age', 'SibSp',
                                                    'Parch', 'Fare'])),
                ('imputer', SimpleImputer(strategy='median'))])

Będziemy także potrzebować imputera do kategorycznych kolumn  napisowych (zwykły Imputer nie działa na tych kolumnach):

In [27]:
# Inspired from stackoverflow.com/questions/25239958
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

Teraz możemy zbudować **pipeline** dla atrybutów kategorycznych:

In [56]:
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
        ("select_cat", DataFrameSelector(["Sex", "Embarked"])),
        ("imputer", MostFrequentImputer()),
        ("cat_encoder", OneHotEncoder(sparse=False, handle_unknown = 'ignore')),
    ])

cat_pipeline.fit(X_train)

Pipeline(steps=[('select_cat',
                 DataFrameSelector(attribute_names=['Sex', 'Embarked'])),
                ('imputer', MostFrequentImputer()),
                ('cat_encoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

In [53]:
# Inspired from stackoverflow.com/questions/25239958
class NameTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return pd.DataFrame(X["Name"].map(lambda x: x.split(", ")[1].split(".")[0]))

In [55]:
from sklearn.preprocessing import OneHotEncoder

rank_pipeline = Pipeline([
        ("select_rank", DataFrameSelector(["Name"])),
        ("imputer", NameTransformer()),
        ("rank_encoder", OneHotEncoder(sparse=False, handle_unknown = 'ignore')),
    ])

rank_pipeline.fit(X_train)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [75]:
# Inspired from stackoverflow.com/questions/25239958
class CabinTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return pd.DataFrame(X["Cabin"].isnull().astype(int))

In [78]:
cabin_pipeline = Pipeline([
        ("select_rank", DataFrameSelector(["Cabin"])),
        ("imputer", CabinTransformer()),
    ])

cabin_pipeline.fit(X_train)

Pipeline(steps=[('select_rank', DataFrameSelector(attribute_names=['Cabin'])),
                ('imputer', CabinTransformer())])

In [79]:
from sklearn.pipeline import FeatureUnion
preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("rank_pipeline", rank_pipeline),
        ("cabin_pipeline", cabin_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

Na koniec połączmy powyższe podejścia:

Teraz mamy fajny **pipeline** przetwarzania wstępnego, który pobiera dane wejściowe i zwraca dane wyjściowe złorzone z liczb, które możemy podać do dowolnego modelu uczenia maszynowego.

In [81]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=10)

In [82]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [83]:
from sklearn.svm import LinearSVC

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('scaler', StandardScaler()),
    ('classifier', LinearSVC())])

param_grid = {
    "scaler": [StandardScaler(), None],
    "classifier__C": [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

grid_1 = GridSearchCV(pipe, param_grid, cv=kfold, scoring='accuracy')
grid_1.fit(X_train, y_train)
grid_1.best_params_

c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.w

{'classifier__C': 0.01, 'scaler': StandardScaler()}

In [105]:
from sklearn.svm import SVC

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('scaler', StandardScaler()),
    ('classifier', SVC(kernel='rbf'))])

param_grid = {
    "scaler": [StandardScaler(), None],
    "classifier__C": [0.1, 1, 10, 100, 1000, 10_000],
    "classifier__gamma": [0.00001, 0.0001, 0.001, 0.01, 0.1, 1]
}

grid_2 = GridSearchCV(pipe, param_grid, cv=kfold, scoring='accuracy')
grid_2.fit(X_train, y_train)
grid_2.best_params_

{'classifier__C': 1000, 'classifier__gamma': 0.001, 'scaler': StandardScaler()}

In [85]:
from sklearn.tree import DecisionTreeClassifier

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('scaler', StandardScaler()),
    ('classifier', DecisionTreeClassifier())])

param_grid = {
    "scaler": [StandardScaler(), None],
    "classifier__criterion": ['gini', 'entropy'],
    "classifier__splitter": ['best', 'random'],
    "classifier__min_samples_split": [5, 10, 15],
}

grid_3 = GridSearchCV(pipe, param_grid, cv=kfold, scoring='accuracy')
grid_3.fit(X_train, y_train)
grid_3.best_params_

{'classifier__criterion': 'entropy',
 'classifier__min_samples_split': 15,
 'classifier__splitter': 'random',
 'scaler': StandardScaler()}

In [112]:
from sklearn.ensemble import VotingClassifier

clf1 = LinearSVC(C=0.01)
clf2 = SVC(kernel="rbf", C=1000, gamma=0.001)
clf3 = DecisionTreeClassifier(criterion="entropy", min_samples_split=15, splitter="random")

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('scaler', StandardScaler()),
    ('classifier', VotingClassifier(estimators=[("linear", clf1), ("rbf", clf2), ("tree", clf3)]))
    ])

param_grid = {
    "scaler": [StandardScaler(), None],
    "classifier__voting": ["soft", "hard"],
}

grid_4 = GridSearchCV(pipe, param_grid, cv=kfold, scoring='accuracy')
grid_4.fit(X_train, y_train)
grid_4.best_params_

c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_scorer.py", line 216, in __call__
    return self._score(
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_scorer.py", line 258, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "c:\Users\

{'classifier__voting': 'hard', 'scaler': StandardScaler()}

# Zad

Robimy StratifiedKFold i znajdujemy optymalne parametry dla

* SVM z jądrem rbf
* SVM z jądrem poly
* SVM liniowego
* Regresji logistycznej

In [113]:
from sklearn import  metrics


models = []
models.append(('SVM linear', grid_1.best_score_))
models.append(('SVM RBF', grid_2.best_score_))
models.append(('Decision Tree', grid_3.best_score_))
models.append(('voting', grid_4.best_score_))

precision_score = []
recall_score = []
f1_score = []
accuracy_score = []
names = []
scores = []
for name, score in models:
    print(name)
    print(score)
    # print("precision_score: {}".format(metrics.precision_score(y_test, model.predict(X_test)) ))
    # print("recall_score: {}".format( metrics.recall_score(y_test, model.predict(X_test)) ))
    # print("f1_score: {}".format( metrics.f1_score(y_test, model.predict(X_test)) ))
    # print("accuracy_score: {}".format( metrics.accuracy_score(y_test, model.predict(X_test)) ))
    names.append(name)
    scores.append(score)
    # precision_score.append(metrics.precision_score(y_test, model.predict(X_test)))
    # recall_score.append(metrics.recall_score(y_test, model.predict(X_test)))
    # f1_score.append( metrics.f1_score(y_test, model.predict(X_test)))
    # accuracy_score.append(metrics.accuracy_score(y_test, model.predict(X_test)))

SVM linear
0.8294257178526842
SVM RBF
0.8372534332084894
Decision Tree
0.8226716604244693
voting
0.8350187265917602


In [114]:
import pandas as pd
d = {
     'accuracy_score' : scores
    }
df = pd.DataFrame(data=d)
df.insert(loc=0, column='Method', value=names)
df

,Method,accuracy_score
0,SVM linear,0.829426
1,SVM RBF,0.837253
2,Decision Tree,0.822672
3,voting,0.835019


In [115]:
clf1 = LinearSVC(C=0.01)
clf2 = SVC(kernel="rbf", C=1000, gamma=0.001)
clf3 = DecisionTreeClassifier(criterion="entropy", min_samples_split=15, splitter="random")

pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('scaler', StandardScaler()),
    ('classifier', VotingClassifier(estimators=[("linear", clf1), ("rbf", clf2), ("tree", clf3)], voting="hard"))
    ])

best_clf = pipe.fit(X_train, y_train)
y_pred = pipe.predict(test_data)

In [116]:
best_estimator = grid_2.best_estimator_

# y_pred = best_estimator.predict(test_data)
result = pd.concat([test_data["PassengerId"], pd.DataFrame({"Survived": y_pred})], axis=1)
result.to_csv("result.csv", index=False)
result

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
